In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-251084
Azure region: northcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-251084


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "CPU-Cluster23"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found Existing Cluster, use it.")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-01-30T10:41:34.913000+00:00', 'errors': None, 'creationTime': '2024-01-30T10:41:26.000432+00:00', 'modifiedTime': '2024-01-30T10:41:36.291560+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.01,0.1, 1),
        '--max_iter': choice(25, 40 ,60, 100, 150, 200)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                script='train.py',
                compute_target=cluster_name,
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=200,
                max_concurrent_runs=4)


In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e140d25b-827d-4e88-8fdc-cc61ac238e13
Web View: https://ml.azure.com/runs/HD_e140d25b-827d-4e88-8fdc-cc61ac238e13?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-251084/workspaces/quick-starts-ws-251084&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-01-30T11:09:33.122370][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-01-30T11:09:33.5949592Z][SCHEDULER][INFO]Scheduling job, id='HD_e140d25b-827d-4e88-8fdc-cc61ac238e13_0' 
[2024-01-30T11:09:33.7376829Z][SCHEDULER][INFO]Scheduling job, id='HD_e140d25b-827d-4e88-8fdc-cc61ac238e13_1' 
[2024-01-30T11:09:33.8323594Z][SCHEDULER][INFO]Scheduling job, id='HD_e140d25b-827d-4e88-8fdc-cc61ac238e13_2' 
[2024-01-30T11:09:33.9696829Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_e140d25b-827d-4e88-8fdc-cc61ac238e13_1' 
[2024-01-30T11:09:33.9513065Z][SCHEDULER][INFO]Scheduling job, id='HD_e140d25b-827d-4e88-8fdc-cc61ac238e13_3

{'runId': 'HD_e140d25b-827d-4e88-8fdc-cc61ac238e13',
 'target': 'CPU-Cluster23',
 'status': 'Completed',
 'startTimeUtc': '2024-01-30T11:09:32.27098Z',
 'endTimeUtc': '2024-01-30T11:17:36.696842Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'd6140234-9faa-4ee0-9033-286b7cc764ad',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '18',
  'best_child_run_id': 'HD_e140d25b-827d-4e88-8fdc-cc61ac238e13_2',
  'score': '0.9088012139605464',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_e140d25b-827d-4e88-8fdc-cc61ac238e13_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClient

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)


Run(Experiment: udacity-project,
Id: HD_e140d25b-827d-4e88-8fdc-cc61ac238e13_2,
Type: azureml.scriptrun,
Status: Completed)


In [9]:
best_run.get_file_names()


['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-01-30-11',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [10]:
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')
print(best_run.get_details()['runDefinition']['arguments'])

print(best_run.get_metrics())

['--C', '0.1', '--max_iter', '100']
{'Regularization Strength:': 0.1, 'Max iterations:': 100, 'Accuracy': 0.9088012139605463}


In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [12]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [14]:
# Submit your automl run

experiment = Experiment(ws, 'automl')
run = experiment.submit(config=automl_config, show_out=True)

2024-01-30:11:20:06,771 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_de2d16e1-2adb-4591-88ef-6263bdc50eb9,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2024/01/30 11:27:47 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024-01-30:11:53:27,442 INFO     [explanation_client.py:334] Using default datastore for uploads


In [16]:
# Retrieve and save your best automl model.

best_auto_run, model = run.get_output()
best_auto_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_auto_run)



Run(Experiment: automl,
Id: AutoML_de2d16e1-2adb-4591-88ef-6263bdc50eb9_20,
Type: None,
Status: Completed)


In [17]:
print(best_auto_run.get_details())

{'runId': 'AutoML_de2d16e1-2adb-4591-88ef-6263bdc50eb9_20', 'target': 'local', 'status': 'Completed', 'startTimeUtc': '2024-01-30T11:51:07.609824Z', 'endTimeUtc': '2024-01-30T11:51:34.953093Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl\',\'compute_target\':\'local\',\'subscription_id\':\'d4ad7261-832d-46b2-b093-22156001df5b\',\'region\':\'northcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_de2d16e1-2adb-4591-88ef-6263bdc50eb9_20","experiment_name":null,"workspace_name":"quick-starts-ws-251084","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_gr

In [18]:
print(best_auto_run.get_metrics())

{'weighted_accuracy': 0.9579786617412704, 'norm_macro_recall': 0.500983981836659, 'f1_score_micro': 0.9166919575113809, 'AUC_macro': 0.9467816687595956, 'f1_score_macro': 0.7721208597066225, 'precision_score_micro': 0.9166919575113809, 'recall_score_micro': 0.9166919575113809, 'precision_score_macro': 0.8000374089945623, 'precision_score_weighted': 0.9108164838469722, 'matthews_correlation': 0.5482834688352942, 'average_precision_score_weighted': 0.9548992056480012, 'accuracy': 0.9166919575113809, 'AUC_micro': 0.9805417635125644, 'f1_score_weighted': 0.9129521133801828, 'average_precision_score_micro': 0.9813218831660828, 'AUC_weighted': 0.9467816687595956, 'log_loss': 0.29893070550762385, 'balanced_accuracy': 0.7504919909183295, 'recall_score_weighted': 0.9166919575113809, 'average_precision_score_macro': 0.822829651372442, 'recall_score_macro': 0.7504919909183295, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_de2d16e1-2adb-4591-88ef-6263bdc50eb9_20/accuracy_table', 'c

In [ ]:
compute_target.delete()